In [1]:
# necessary packages #
#cd("./Documents/Github/Multi_NNGP/RDA/projects/misalign_response/")
#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using DataFrames
using Dates
#using NearestNeighbors # Can't use KDtree in julia for posterior prediction

In [2]:
include("../../../util2.j")

colnorm (generic function with 1 method)

In [3]:
# unnecessary packages #

#using Pkg
#Pkg.add("UnicodePlots")
using UnicodePlots   # check the structure of the sparse matrix
using BenchmarkTools

using StatsPlots
using MCMCChains
using PrettyTables

In [4]:
#using Pkg
#Pkg.add("NearestNeighborDescent");
#using NearestNeighborDescent

In [5]:
Sys.free_memory()/(2^20*1024)

20.33291244506836

In [6]:
@time @load "../../data/LMC_data/misalign_res_data.jld"

┌ Warning: type OrderedCollections.OrderedDict{Symbol,Any} does not exist in workspace; reconstructing
└ @ JLD2 /home/lu/.julia/packages/JLD2/KjBIK/src/data.jl:1153


 15.175661 seconds (8.04 M allocations: 2.036 GiB, 5.99% gc time)


39-element Array{Symbol,1}:
 :N            
 :U_indx       
 :U_ind        
 :S_indx       
 :S_ind        
 :M_ind        
 :M            
 :R            
 :NR           
 :NS           
 :NM           
 :NU           
 :coords_total 
 ⋮             
 :parts        
 :parts_train  
 :parts_test   
 :CVNN_train   
 :CVNN_test    
 :CVnnIndx_col 
 :CVnnIndx_row 
 :CVMnnIndxLU  
 :CVMnnIndx    
 :CVMnndists   
 :CVIndex_train
 :CVRSMPEind   

In [7]:
using RData
@time data_cleaned = load("../../data/rawdata/data_cleaned_small_expanded.RData", convert = true)["data_cleaned_small"];
@time hold_index_set = load("../../data/rawdata/hold_index_small_expanded.RData", convert = true);

  9.986620 seconds (41.91 M allocations: 2.489 GiB, 2.51% gc time)
  0.068155 seconds (55.17 k allocations: 7.249 MiB)


In [8]:
# pick explanatory variables 12 to 24, no 16
DataFrames.describe(data_cleaned)

,variable,mean,min,median,max
,Symbol,Float64,Float64,Float64,Float64
1,x,-1.04265e7,-1.11193e7,-1.03937e7,-1.00078e7
2,y,3.83786e6,3.33608e6,3.81839e6,4.44757e6
3,scaled_x,0.6928,0.0,0.725548,1.11149
4,scaled_y,0.501779,0.0,0.482309,1.11149
5,NDVI,0.240122,-0.00954541,0.192437,0.666598
6,EVI,0.150172,-0.00400802,0.122129,0.605463
7,red_reflectance,0.144424,0.0047,0.1452,0.3892
8,NIR_reflectance,0.243485,0.0116,0.2383,0.6165
9,blue_reflectance,0.0752273,0.0001,0.0755,0.2


In [9]:
Y_ord = convert(Matrix, data_cleaned[:, [:NDVI, :red_reflectance]]);
X_ord = dropzeros(sparse([fill(1.0, (N, 1)) convert(Matrix, data_cleaned[:, 28:28])]));

In [10]:
# priors #
p = 2; q = 2;
μβ = fill(0.0, p, q); inv_Vr = spzeros(p, p); # set Vr^{-1} be zero for the simulation...
νΣ = q + 1; ΨΣ = [[1.0 0.0]; [0.0 1.0]];
inv_Lr = spzeros(p, p); Lμβ = inv_Lr * μβ;

In [19]:
# (a) grid of candidate value
ϕ_grid = vcat(LinRange(3 / sqrt(2), 3 * 100 / sqrt(2), 10))
#LinRange(3 / sqrt(2), 3 * 100 / sqrt(2), 6) LinRange(2, 86, 6)LinRange(35.6, 69.2, 6)
α_grid = vcat(LinRange(0.6, 0.999, 20)) 
#LinRange(0.9, 0.99, 4) LinRange(0.96, 0.999, 4) 

20-element Array{Float64,1}:
 0.6               
 0.621             
 0.642             
 0.6629999999999999
 0.6839999999999999
 0.7050000000000001
 0.726             
 0.7469999999999999
 0.768             
 0.789             
 0.8099999999999999
 0.831             
 0.852             
 0.873             
 0.894             
 0.9149999999999999
 0.936             
 0.957             
 0.978             
 0.999             

In [56]:
ϕ_grid

10-element Array{Float64,1}:
   2.1213203435596424
  25.455844122715707 
  48.790367901871775 
  72.12489168102783  
  95.4594154601839   
 118.79393923933998  
 142.12846301849603  
 165.4629867976521   
 188.79751057680818  
 212.13203435596424  

In [13]:
using Base.Threads
println("Number of threads = $(nthreads())")

Number of threads = 4


In [14]:
Dates.format(now(), "HH:MM:SS") 

"12:37:16"

In [ ]:
# Threads.@threads for i in 1:kf
#     print(Threads.threadid())
# end

In [ ]:
# For loop
CVRMSPE_i = fill(0.0, length(ϕ_grid), length(α_grid), kf);
ns = length(S_ind[1]) + length(S_ind[2]);
prog = Progress(kf, 1, "Computing initial pass...", 50)
@time Threads.@threads for i in 1:kf
    print(Threads.threadid())
    CVnIndx = length(CVNN_train[i].nnIndx);
    nk = length(parts_test[i]);
    nr = length(CVIndex_train[i]);
    CVA = Array{Float64}(undef, CVnIndx); CVD = Array{Float64}(undef, nr);
    CVDI_AX = []; CVDI_AY = []; CVVstar = []; CVμstar = [];
    
    CVAtilde = Array{Float64}(undef, nk * m); CVDtilde = Array{Float64}(undef, nk);
    CVAtildeM = [];
    
@time    for j in 1:length(ϕ_grid) 
        for k in 1:length(α_grid)
            
            # obtain posterior estimators of beta
            getAD_collapse(coords_total[CVIndex_train[i],:], 
                CVNN_train[i].nnIndx, CVNN_train[i].nnDist, CVNN_train[i].nnIndxLU, 
                ϕ_grid[j], 0.5, α_grid[k], CVA, CVD);
            CVDI_AX = Diagonal(1 ./ sqrt.(CVD)) * (sparse(CVnnIndx_row[i], CVnnIndx_col[i], 
                    vcat(-CVA, ones(nr))) * X_ord[CVIndex_train[i], :]);
            CVDI_AY = Diagonal(1 ./ sqrt.(CVD)) * (sparse(CVnnIndx_row[i], CVnnIndx_col[i], 
                    vcat(-CVA, ones(nr))) * Y_ord[CVIndex_train[i], :]);
            CVVstar = inv(Symmetric(Matrix(CVDI_AX'CVDI_AX + inv_Vr)));
            CVμstar = CVVstar * (CVDI_AX'CVDI_AY + inv_Vr * μβ);
            
            # obtain posterior predictions over holded folder
            getAD_collapse(coords_total[CVIndex_train[i], :], CVMnnIndx[i], 
                CVMnndists[i], CVMnnIndxLU[i],  ϕ_grid[j], 0.5, α_grid[k], CVAtilde, CVDtilde)
            CVAtildeM = sparse(repeat(1:nk, inner = m), CVMnnIndx[i], CVAtilde, nk, nr);
            Y_hold_mean = X_ord[S_indx[parts_test[i]], :] * CVμstar + 
                CVAtildeM * (Y_ord[CVIndex_train[i], :] - X_ord[CVIndex_train[i], :] * CVμstar);
            
            # Calculate RMSPE over observed responses
            for l in 1:q
                CVRMSPE_i[j, k, i] = CVRMSPE_i[j, k, i] + mean((Y_ord[S_indx[parts_test[i]], l] - 
                        Y_hold_mean[:, l])[CVRSMPEind[i, l]].^2) * length(CVRSMPEind[i, l]) / ns;
            end
        end
    end
#    next!(prog) # monitor the progress
end
CVRMSPE = [sqrt(sum(CVRMSPE_i[i, j, :])) for i in 1:length(ϕ_grid), j in 1:length(α_grid)]

25143

3273.851766 seconds (14.68 G allocations: 4.612 TiB, 50.52% gc time)

3278.755131 seconds (14.70 G allocations: 4.620 TiB, 50.53% gc time)

3345.083753 seconds (15.00 G allocations: 4.713 TiB, 50.46% gc time)

3376.256816 seconds (15.12 G allocations: 4.754 TiB, 50.37% gc time)

3390.717529 seconds (15.17 G allocations: 4.768 TiB, 50.27% gc time)

3391.677044 seconds (15.17 G allocations: 4.769 TiB, 50.25% gc time)


In [31]:
Dates.format(now(), "HH:MM:SS") 

"12:35:39"

In [ ]:
# cross-validation for choosing delta and phi ...
pick_ind = argmin(CVRMSPE);
ϕ_pick = ϕ_grid[pick_ind[1]]; α_pick = α_grid[pick_ind[2]]; # alpha ranges from 0 to 1. alpha = sigma2 / (sigma2 + tau2)
[ϕ_pick, α_pick]

In [ ]:
using RCall
@rput CVRMSPE
@rput ϕ_grid α_grid
R"""
library(ggplot2)
library(hrbrthemes)

# Dummy data
x <- round(ϕ_grid, digits = 4)
y <- round(α_grid, digits = 6)
data <- expand.grid(X=x, Y=y)
trun_ind = which(CVRMSPE > 0.02570);
CVRMSPE[trun_ind] = 0.02570 #0.021025
data$Z <- c(CVRMSPE)#[7:19, 6:18] 

width <- 360*1.2
height <- 360
pointsize <- 16


#png(paste("../../pics/conj_latent_RMSPE_map1.png", sep = ""), 
#    width = width, height = height, pointsize = pointsize, family = "Courier")
#par(mfrow = c(1, 1))
# Heatmap 
ggplot(data, aes(X, Y, fill= Z)) + 
  geom_tile() +
  scale_fill_gradient(low="white", high="blue") + theme_ipsum()
#dev.off()

#colnames(CVRMSPE) = round(ϕ_grid, digits = 4);
#rownames(CVRMSPE) = α_grid;
#heatmap(CVRMSPE, Colv = NA, Rowv = NA)
"""

In [11]:
Sys.free_memory()/(2^20*1024)

16.487831115722656

In [12]:
ϕ_pick = 26.414213562373096 
α_pick = 0.9926666666666666

0.9926666666666666

In [13]:
# Prepare for the log-likelihood in MH

# Build μ* V* Ψ* v* for prior
nIndx = length(NN.nnIndx);
A = Array{Float64}(undef, nIndx); D = Array{Float64}(undef, NR);
getAD_collapse(coords_total[R, :], NN.nnIndx, NN.nnDist, NN.nnIndxLU, ϕ_pick, 0.5, α_pick, A, D);
DI_AX = Diagonal(1 ./ sqrt.(D)) * (sparse(nnIndx_row, nnIndx_col, vcat(-A, ones(NR))) * X_ord[R, :]);
DI_AY = Diagonal(1 ./ sqrt.(D)) * (sparse(nnIndx_row, nnIndx_col, vcat(-A, ones(NR))) * Y_ord[R, :]);
Vstar = inv(Symmetric(Matrix(DI_AX'DI_AX + inv_Vr)));
μstar = Vstar * (DI_AX'DI_AY + inv_Vr * μβ);
Lμstar = cholesky(Vstar).L \ μstar;
Ψstar = ΨΣ + DI_AY'DI_AY + Lμβ'Lμβ - Lμstar'Lμstar;
νstar = νΣ + NR;
inv_Vstar = inv(Vstar);
logdet_Vstar = logdet(Vstar);
#Lstar = cholesky(Vstar);

# Construct AMtilde DMtilde for observations on M for updating Σ from prior Σ | YR 
AMtilde = Array{Float64}(undef, NM * m); DMtilde = Array{Float64}(undef, NM);
getAD_collapse(coords_total[R, :], MnnIndx, Mnndists, MnnIndxLU, 
    ϕ_pick, 0.5, α_pick, AMtilde, DMtilde)
AMtildeM = sparse(repeat(1:NM, inner = m), MnnIndx, AMtilde, NM, NR);

A2tilde = Array{Float64}(undef, NU * m); D2tilde = Array{Float64}(undef, NU);
getAD_collapse(coords_total[R, :], MnnIndx2, Mnndists2, MnnIndxLU2, 
    ϕ_pick, 0.5, α_pick, A2tilde, D2tilde)
A2tildeM = sparse(repeat(1:NU, inner = m), MnnIndx2, A2tilde, NU, NR);
D2tildeD = Diagonal(sqrt.(D2tilde));


In [14]:
# Preallocation for posterior samples  of Σ#
N_sam = 10000; N_burnin = Integer(trunc(N_sam * 0.75));
Σ_sam = Array{Float64, 3}(undef, q, q, N_sam + 1);
L_sam = Array{Float64, 3}(undef, q, q, N_sam + 1);
A_sam = Array{Float64, 1}(undef, N_sam); # acceptance rate
llh_sam = Array{Float64, 1}(undef, N_sam + 1); # record log-likelihood 

cholΣ_sam = Array{Float64, 2}(undef, q, q);

#Preallocation for posterior samples of vecβ#
vec_β_sam = Array{Float64, 2}(undef, q * p, N_sam - N_burnin);

#Preallocation for posterior samples of missing value over M#
Y_miss_sam = Array{Float64, 2}(undef, length(M_ind[1]) + length(M_ind[2]), N_sam - N_burnin);

#Preallocation for posterior samples of missing value over U#
Y_U_sam = Array{Float64, 3}(undef, NU, q, N_sam - N_burnin);


In [17]:
# check the prior of Σ | YR
Random.seed!(0);
Σ_prior_sam = rand(InverseWishart(νstar, Ψstar), 500);
L_prior_sam = [cholesky(Σ_prior_sam[i]).L[j] for i in 1:500, j in 1:(q * q)];
#Σ_cor_prior_sam_trans = tan.(π / 2 * Σ_cor_prior_sam); # we update the transformed value: tan(π/2 * cor) in MH
Σ_sam[:, :, 1] = Ψstar / (νstar - q - 1);
L_sam[:, :, 1] = cholesky(Σ_sam[:, :, 1]).L
scale_L = cholesky(cov(L_prior_sam[:, [1, 2, 4]]) * 2.38^2).L;

In [18]:
scale_L

3×3 LowerTriangular{Float64,Array{Float64,2}}:
  0.000352819   ⋅           ⋅         
 -0.000118615  0.00020996   ⋅         
 -1.76039e-6   9.94873e-6  0.000150364

In [19]:
# build H2 H1 YM_mean and vec(YM-YM_mean)

M_sort = fill(0, N, q);
M_sort[S_ind[1], 1] .= 1; M_sort[S_ind[2], 2] .= 1;
M_sort_index = (M_sort[:, 1] * (2^0) +  M_sort[:, 2] * (2^1))[M];
Dict_obs = Dict(2^0 => [1], 2^1 => [2], (2^0 + 2^1)=> [1, 2]);
M_obs_index = [any(x->x==j, Dict_obs[M_sort_index[i]]) for i in 1:NM, j in 1:q];
M_obs_index2 = [Dict_obs[M_sort_index[i]] for i in 1:NM];
Dict_miss = Dict(2^0 => [2], 2^1 => [1], 0 => [1, 2]);
M_miss_index = [Dict_miss[M_sort_index[i]] for i in 1:NM];
L = sum(M_sort[M, :]); # in this case L = NM
H1 = fill(0.0, L, q * p);
H2 = spzeros(L, q * NR);
vec_YM = zeros(L);
DMl = zeros(L);
for j in 1:q
    H1[collect(1:L)[M_obs_index[:, j]], (j - 1) * p .+ (1:p)] = 
        X_ord[M[M_obs_index[:, j]], :];
    H2[collect(1:L)[M_obs_index[:, j]], (j - 1) * NR .+ (1:NR)] = 
        AMtildeM[M_obs_index[:, j], :];
    vec_YM[collect(1:L)[M_obs_index[:, j]]] = Y_ord[M[M_obs_index[:, j]], j];
    DMl[collect(1:L)[M_obs_index[:, j]]] = DMtilde[M_obs_index[:, j]]; 
end
H1 = H1 - H2 * kron(Diagonal(ones(q)), X_ord[R, :]);
YM_mean = H1 * vec(μstar) + H2 * vec(Y_ord[R, :]);
vec_YM_YM_mean = vec_YM - YM_mean;
Ystar = vcat(vec(DI_AY), (vec_YM - H2 * vec(Y_ord[R, :])) ./ (sqrt.(DMl)), vec(Lμβ));
Xstar = vcat(kron(Diagonal(ones(q)), DI_AX), Diagonal(1 ./ sqrt.(DMl)) * H1, kron(Diagonal(ones(q)), inv_Lr));

In [21]:
Dates.format(now(), "HH:MM:SS") 

"17:14:57"

In [22]:
# MCMC algorithm for obtaining posterior sample of Σ#
Random.seed!(123);
prog = Progress(N_sam, 1, "Computing initial pass...", 50)
for l in 1:N_sam
    if l == 1
        cholΣ_sam = cholesky(Σ_sam[:, :, l]);
        Dic_diag = Dict(2^0 => sparse(1I, 1, 1) * (1 / (Σ_sam[:, :, l][1, 1])), 
            2^1 => sparse(1I, 1, 1) * (1 / (Σ_sam[:, :, l][2, 2])));
        invD_ele = [x for i in 1:NM for x in (Dic_diag[M_sort_index[i]] ./ DMtilde[i])];
        invD = sparse(1:NM, 1:NM, invD_ele);
        invD_half = sparse(1:NM, 1:NM, sqrt.(invD_ele));
        logdetD = -sum([log(det(Dic_diag[M_sort_index[i]] ./ DMtilde[i]))  for i in 1:NM]);
        inv_V_prior = kron(inv(cholΣ_sam), inv_Vstar);
        logdet_V_prior = p * logdet(cholΣ_sam) + q * logdet_Vstar;
        invD_half_H1 = invD_half * H1;
        chol_V_update = cholesky(Symmetric(inv_V_prior + invD_half_H1'invD_half_H1));
        u = chol_V_update.L \ collect(vec_YM_YM_mean'invD*H1)[1:size(chol_V_update.L)[2]];
        llh_sam[1] = logpdf(InverseWishart(νstar, Ψstar), Σ_sam[:, :, l]) - 
            0.5 * (logdet(chol_V_update) + logdet_V_prior + logdetD + 
            norm(invD_half * vec_YM_YM_mean)^2 - norm(u)^2) +  
        sum([(q - i + 1) * log(abs(L_sam[i, i, 1])) for i in 1:q]);
    end
    
    
    # use metropolis-hasting to update range
    v_old = [L_sam[1, 1, l], L_sam[2, 1, l], L_sam[2, 2, l]];
    v_new = v_old + scale_L * rand(Normal(), 3);
    L_sam[1, 1, l + 1] = v_new[1];
    L_sam[2, 1, l + 1] = v_new[2];
    L_sam[2, 2, l + 1] = v_new[3];
    L_sam[1, 2, l + 1] = 0.0;
    Σ_sam[:, :, l + 1] = Symmetric(L_sam[:, :, l + 1] * transpose(L_sam[:, :, l + 1]));
    cholΣ_sam = cholesky(Σ_sam[:, :, l + 1]);
    Dic_diag = Dict(2^0 => sparse(1I, 1, 1) * (1 / (Σ_sam[:, :, l + 1][1, 1])), 
        2^1 => sparse(1I, 1, 1) * (1 / (Σ_sam[:, :, l + 1][2, 2])));
    invD_ele = [x for i in 1:NM for x in (Dic_diag[M_sort_index[i]] ./ DMtilde[i])];
    invD = sparse(1:NM, 1:NM, invD_ele);
    invD_half = sparse(1:NM, 1:NM, sqrt.(invD_ele));
    #invD = blockdiag([Dic_diag[M_sort_index[i]] ./ DMtilde[i]  for i in 1:NM ]...);
    logdetD = - sum([log(det(Dic_diag[M_sort_index[i]] ./ DMtilde[i]))  for i in 1:NM]);
    inv_V_prior = kron(inv(cholΣ_sam), inv_Vstar);
    logdet_V_prior = p * logdet(cholΣ_sam) + q * logdet_Vstar;
    invD_half_H1 = invD_half * H1;
    chol_V_update = cholesky(Symmetric(inv_V_prior + invD_half_H1'invD_half_H1));
    u = chol_V_update.L \  collect(vec_YM_YM_mean'invD * H1)[1:size(chol_V_update.L)[2]];
    llh_sam[l + 1] = logpdf(InverseWishart(νstar, Ψstar), Σ_sam[:, :, l + 1]) - 
        0.5 * (logdet(chol_V_update) + logdet_V_prior + logdetD + 
        norm(invD_half * vec_YM_YM_mean)^2 - norm(u)^2) + 
        sum([(q - i + 1) * log(abs(L_sam[i, i, l + 1])) for i in 1:q]);
    A_sam[l] = min(exp(llh_sam[l + 1] - llh_sam[l]), 1.0); # acceptance rate
    if rand(1)[1] > A_sam[l]
       # Don't update
        L_sam[:, :, l + 1] = L_sam[:, :, l]; 
        Σ_sam[:, :, l + 1] = Σ_sam[:, :, l]; 
        llh_sam[l + 1] = llh_sam[l];
    end  
    next!(prog) # monitor the progress
end

Computing initial pass...100%|██████████████████████████████████████████████████| Time: 1:02:12


| Time: 1:02:12

In [23]:
mean(A_sam)

0.1165849170589811

0.1165849170589811

In [24]:
N

1020000

In [27]:
NR

816465

In [26]:
NU

22146

In [28]:
# generate the posterior samples of β and prediction over S with missing observations and over tested locations#
Random.seed!(123);
prog = Progress(N_sam - N_burnin, 1, "Computing initial pass...", 50)
for l in (N_burnin + 1):N_sam
    
    cholΣ_sam = cholesky(Σ_sam[:, :, l]);
    Dic_diag = Dict(2^0 => sparse(1I, 1, 1) * (1 / (Σ_sam[:, :, l][1, 1])), 
        2^1 => sparse(1I, 1, 1) * (1 / (Σ_sam[:, :, l][2, 2])), 
        (2^0 + 2^1)=> sparse(inv(cholΣ_sam)));
    invD_ele = [x for i in 1:NM for x in (Dic_diag[M_sort_index[i]] ./ DMtilde[i])];
    invD = sparse(1:NM, 1:NM, invD_ele);
    inv_Vη = blockdiag(kron(Dic_diag[(2^0 + 2^1)], sparse(1I, NR, NR)), 
            invD, kron(Dic_diag[(2^0 + 2^1)], sparse(1I, p, p))); 
    invD_half = sparse(1:NM, 1:NM, sqrt.(invD_ele));
    inv_Vη_half = blockdiag(kron(sqrt(inv(cholΣ_sam)), sparse(1I, NR, NR)), 
            invD_half, kron(sqrt(inv(cholΣ_sam)), sparse(1I, p, p))); 
    inv_Vη_half_Xstar = inv_Vη_half * Xstar;
    chol_inv_Vβstar = cholesky(Symmetric(inv_Vη_half_Xstar'inv_Vη_half_Xstar));
    vec_β_sam[:, l - N_burnin] = chol_inv_Vβstar.U \ (chol_inv_Vβstar.L \ 
        (collect(Ystar'inv_Vη*Xstar)[1:size(chol_inv_Vβstar.L)[2]]) +
        rand(Normal(), p * q));
    # update posterior predictions over S with missing observations
    β_sam = reshape(vec_β_sam[:, l - N_burnin], p, q);
    MeanYM = X_ord[M, :] * β_sam + AMtildeM * (Y_ord[R, :] - X_ord[R, :] * β_sam);
    Dic_diag_cond_std = Dict(2^0 => sqrt(Σ_sam[2, 2, l] - Σ_sam[2, 1, l] / Σ_sam[:, :, l][1, 1] * Σ_sam[1, 2, l]), 
        2^1 => sqrt(Σ_sam[1, 1, l] - Σ_sam[1, 2, l] / Σ_sam[2, 2, l] * Σ_sam[2, 1, l]));
    Dic_diag_cond_cor = Dict(2^0 => Σ_sam[2, 1, l] / Σ_sam[1, 1, l], 
        2^1 => Σ_sam[1, 2, l] / Σ_sam[2, 2, l]);
    j = 1;
    for i in 1:NM
        Y_miss_sam[j:(j + length(M_miss_index[i]) - 1), l - N_burnin] = 
            MeanYM[i, M_miss_index[i]] + Dic_diag_cond_cor[M_sort_index[i]] *
            (Y_ord[M[i], M_obs_index2[i]] - MeanYM[i, M_obs_index2[i]]) + 
            sqrt(DMtilde[i]) * rand(Normal(0, Dic_diag_cond_std[M_sort_index[i][1]]), 1);
        j = j + length(M_miss_index[i]);
    end
    Y_U_sam[:, :, l - N_burnin] = X_ord[U_indx, :] * β_sam + A2tildeM * (Y_ord[R, :] - X_ord[R, :] * β_sam) + 
        D2tildeD * rand(Normal(), NU, q) * cholΣ_sam.U;
    
    next!(prog) # monitor the progress
end

Computing initial pass...100%|██████████████████████████████████████████████████| Time: 1:12:39


| Time: 1:12:39

In [29]:
Y_m_pos_qt = Array{Float64, 2}(undef, length(M_ind[1]) + length(M_ind[2]), 3);
Y_m_pos_mean = Array{Float64, 1}(undef, length(M_ind[1]) + length(M_ind[2]));
Y_m_pos_var = Array{Float64, 1}(undef, length(M_ind[1]) + length(M_ind[2]));
for i in 1:(length(M_ind[1]) + length(M_ind[2]))
    Y_m_pos_qt[i, :] = quantile(Y_miss_sam[i, 1:(N_sam - N_burnin)], [0.025, 0.5, 0.975]);
    Y_m_pos_mean[i] = mean(Y_miss_sam[i, 1:(N_sam - N_burnin)])
    Y_m_pos_var[i] = var(Y_miss_sam[i, 1:(N_sam - N_burnin)])
end
Y_U_pos_qt = Array{Float64, 3}(undef, NU, q, 3);
Y_U_pos_mean = Array{Float64, 2}(undef, NU, q);
Y_U_pos_var = Array{Float64, 2}(undef, NU, q);
for j in 1:q
    for i in 1:NU
        Y_U_pos_qt[i, j, :] = quantile(Y_U_sam[i, j, 1:(N_sam - N_burnin)], [0.025, 0.5, 0.975]);
        Y_U_pos_mean[i, j] = mean(Y_U_sam[i, j, 1:(N_sam - N_burnin)]);
        Y_U_pos_var[i, j] = var(Y_U_sam[i, j, 1:(N_sam - N_burnin)]);
    end
end

In [30]:
@save "../../results/misalign_resp/misalign_resp_results.jld" N_sam N_burnin vec_β_sam Σ_sam Y_m_pos_qt Y_m_pos_mean Y_m_pos_var Y_ord M_miss_index Y_U_pos_qt Y_U_pos_mean Y_U_pos_var ϕ_pick α_pick